In [1]:
import redis
import json

In [2]:
REDIS = redis.Redis(host='data_store')

In [10]:
#Test
REDIS.set('my_incrementer', 42)

True

In [5]:
REDIS.get('IEX_Stock_Count')

In [ ]:
REDIS.get('Twitter_Stock_Count')

In [5]:
#Manager needs to do many things. One of which is manage the the company list and populate the data-store
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }
REDIS.set('companies', json.dumps(companies))

True

In [6]:
#TEst
REDIS.set('test',1)

True

In [11]:
int(REDIS.get('test')) ==1

True

In [12]:
#When setting up the flag, need to setup scheudle for the time, but also need to check if after open time
# and before close time, and set the flag to true.

In [19]:
!pip install tzlocal

  Running setup.py bdist_wheel for tzlocal ... done
  Stored in directory: /home/ds/.cache/pip/wheels/7c/a1/5d/0f37ce6eb6eea391bd185f5747429a93519be115d007263bcf
Successfully built tzlocal


In [6]:
#Test the publish subscribe
p = REDIS.pubsub()

In [7]:
p.subscribe('test-channel')

In [16]:
#Use the same json dumps as setting the companies.
payload = json.dumps({'event':'test'})
REDIS.publish('test-channel', payload)
#REDIS.publish('test-channel', 'other data')

1

In [9]:
#Once publish teh data, can only read the data once. Once its read its gone.
#Have to continuously pull off the queue.
p.get_message()

{'channel': b'test-channel', 'data': 1, 'pattern': None, 'type': 'subscribe'}

In [11]:
if p.get_message():
    print('Data There')
else:
    print('Data Gone')

Data There


In [17]:
#To retreive data.
message = p.get_message()
print(message['channel'].decode())
print(json.loads(message['data'].decode()))

test-channel
{'event': 'test'}


Code that needs to be inside the Twitter and the IEX notebooks.

In [4]:
################# This will be in each data source code.
#To setup the queue
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')

#Code to log to the event queue
def send_event(source, kind, message):
    event_time = datetime.datetime.now()
    event = {
            "event_time": event_time,
            "source": source,
            "kind" : kind,
            "message" : message
            }
    payload = json.dumps(event)
    REDIS.publish('event_queue', payload)

def send_log(source, current_count, count_diff):
    log_time = datetime.datetime.now()
    log = {
            "log_time": log_time,
            "source": source,
            "current_count" : current_count,
            "count_diff" : count_diff
            }
    payload = json.dumps(log)
    REDIS.publish('log_queue', payload)

#This functions will replace writing the database in the source files. Need to then write to the datbase in the manager
#How to handle the multiple channels? The get message has which channel. Just need to check it.



### Manager

#### Purpose:
1. Setup the log table
2. Setup the error table
3. Setup the company list and set the Redis Value
4. Control the Sync schedule (When to set the DataOn variable)
5. Subscribe to the error subscription and log errors 

In [1]:
############# Imports ##################################
import redis
import json
import schedule
import time
import datetime
#from tzlocal import get_localzone

import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import Column
from sqlalchemy import Integer, String, DateTime

############# Redis Setup ##############################
#Connect to the DataStore
REDIS = redis.Redis(host='data_store')

#To setup the queue
queue = REDIS.pubsub()
#Subscribe to the queues one for the events and one for the log
queue.subscribe('event_queue')
queue.subscribe('log_queue')

In [2]:
############# Setup Database ###########################
User = %env DB_USER
password = %env DB_PWD
dbname = %env DB_NAME


engine = create_engine('mysql+mysqlconnector://{}:{}@db:3306/{}'.format(User, password, dbname), echo=False)
inspector = inspect(engine)



#Check to see if the tables are created and if not, create them
meta = MetaData(engine)

#Create log table
if not engine.dialect.has_table(engine, 'log'):
    print('Log Table does not exist')
    print('Log Table being created....')
    #Time, Source, Current Count, Count Diff
    t1 = Table('log', meta,
                 Column('log_time', DateTime, default=datetime.datetime.utcnow),
                 Column('Source', String(30)),
                 Column('Current_Count', Integer),
                 Column('Count_Diff', Integer))
    t1.create()
else:
    print('Log Table Exists')

#Create event table
if not engine.dialect.has_table(engine, 'event'):
    print('Event Table does not exist')
    print('Event Table being created....')
    #Time, Source, Current Count, Count Diff
    t1 = Table('event', meta,
                 Column('event_time', DateTime, default=datetime.datetime.utcnow),
                 Column('Source', String(30)),
                 Column('Kind', String(30)),
                 Column('Message', String(8000)))
    t1.create()
else:
    print('Event Table Exists')


    

Log Table Exists
Event Table does not exist
Event Table being created....


In [3]:
################## Define Functions ##############################

#Database Function creation

#Create table object
meta = MetaData(engine,reflect=True)
table = meta.tables['log']

def database_log(log_data):
    #Need to log these items to a database.
    ins = table.insert().values(
            log_time = log_data['log_time'],
            Source = log_data['source'],
            Current_Count=log_data['current_count'],
            Count_Diff=log_data['count_diff']
               )
    conn.execute(ins)
    print('Logged Log Data')
    
#Create table object
meta = MetaData(engine,reflect=True)
table = meta.tables['event']

def database_event(event_data):
    #Need to log these items to a database.
    ins = table.insert().values(
            event_time = event_data['event_time'],
            Source = event_data['source'],
            Kind=event_data['kind'],
            Message=event_data['message']
               )
    conn.execute(ins)
    print('Logged Event Data')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.


In [9]:
#Test insert <DO NOT USE - This is just for testing purposes>
conn = engine.connect()
ins = t1.insert().values(
            log_time = datetime.datetime.now(),
            Source = 'Test',
            Current_Count=42,
            Count_Diff=5
               )
conn.execute(ins)


In [4]:
########### Set up the company list #####################
companies = {
    "AAPL":"Apple",
    "FB":"Facebook",
    "GOOG":"Google"
    }
REDIS.set('companies', json.dumps(companies))

True

In [5]:
########### Setup the Data ON Flag #################
REDIS.set('Data_On',0)

#Create Explicit Data On/Off Functions
def DataOn():
    REDIS.set('Data_On',1)
    

def DataOff():
    REDIS.set('Data_On',0)
    
global past_twitter_count
past_tweet_count = 0

In [15]:
#Turn on if starts during open times
now = datetime.datetime.now()

if now.hour >= 14 and now.hour < 21:
    if now.hour = 14 and now.minute >= 30:
        DataOn()
    elif now.hour > 14:
        DataOn()

17

In [6]:
########## Setup Schedules ######################
#Since the market isnt open all day, want to control when turn data fetch on and off

#Time in 24 hour clocks and UTC time
schedule.clear()
#Setup Data On/Off Schedule
schedule.every().monday.at("14:30").do(DataOn)
schedule.every().monday.at("21:00").do(DataOff)
schedule.every().tuesday.at("14:30").do(DataOn)
schedule.every().tuesday.at("21:00").do(DataOff)
schedule.every().wednesday.at("14:30").do(DataOn)
schedule.every().wednesday.at("21:00").do(DataOff)
schedule.every().thursday.at("14:30").do(DataOn)
schedule.every().thursday.at("21:00").do(DataOff)
schedule.every().friday.at("14:30").do(DataOn)
schedule.every().friday.at("21:00").do(DataOff)


Every 1 week at 21:00:00 do DataOff() (last run: [never], next run: 2018-02-09 21:00:00)

In [12]:
########### Execute ############################

while True:
    schedule.run_pending()
    
    #May need to add in another while loop here. But we shall see after testing.
    
    next_message = queue.get_message()
    #next_message = json.loads(queue.get_message()['data'].decode())
    if next_message:
        payload = next_message['data'].decode()
        #check which queue
        if next_message['channel'].decode() == 'event_queue':
            #Call database_event function to log to database
            database_event(payload)
        
        if next_message['channel'].decode() == 'log_queue':
            #Call database_log function to log to database
            database_log(payload)
    
    time.sleep(1)

TypeError: string indices must be integers